In [ ]:
import cantools
import re
from datetime import datetime

# Function to log errors to a separate file
def log_error(message, line, row_num):
    """
    Log an error message to a separate file, including the raw message and row number.

    Args:
        message (str): The error message to be logged.
        line (str): The raw line that caused the error.
        row_num (int): The row number of the error.
    """
    error_log_file = 'testing_data/CanTraceJuly11_errors.txt'
    log_message = f"Row {row_num}: {message} | Raw Message: {line.strip()}"
    with open(error_log_file, 'a') as f:
        f.write(log_message + '\n')

def parse_can_message(line, db, row_num):
    """
    Parse a CAN message line, decode it, and return a list of formatted strings.

    Args:
        line (str): A single line representing a CAN message.
        db (cantools.database.Database): Loaded CAN database.
        row_num (int): The row number of the line being processed.

    Returns:
        list: A list of formatted strings for each signal in the CAN message.
    """
    pattern = r'\s*(\d+)\s+(\w+)(?:\s+X)?\s+(\d+)\s+([0-9\s]+)\s+(\d+\.\d+)\s+([RX])'
    match = re.match(pattern, line)
    if not match:
        log_error("Invalid format", line, row_num)
        return []

    index, can_id, dlc, data_bytes, timestamp, direction = match.groups()
    can_id_int = int(can_id)

    # Try to get the CAN message definition from the DBC
    try:
        message = db.get_message_by_frame_id(can_id_int)
    except KeyError:
        log_error(f"No message found for CAN ID {can_id_int}", line, row_num)
        return []

    # Parse and decode data bytes
    try:
        data_list = bytes(int(byte) for byte in data_bytes.split() if byte)
        decoded_data = message.decode(data_list)
    except Exception as e:
        log_error(f"Decoding error for CAN ID {can_id_int}: {str(e)}", line, row_num)
        return []

    # Generate formatted strings for each signal
    output = []
    for signal_name, value in decoded_data.items():
        try:
            signal = message.get_signal_by_name(signal_name)
            description = signal.comment if signal.comment else "No description available"
            unit = signal.unit if signal.unit else "N/A"
        except AttributeError:
            description = "No description available"
            unit = "N/A"

        now = datetime.now()
        unix_time_ns = int(now.timestamp() * 1e9)
        formatted_string = (
            f"canBus,"
            f"signalName={signal_name},"
            f"description={description},"
            f"messageName={message.name},"
            f"RawCAN={can_id},"
            f"sensorReading={value},"
            f"unit={unit},"
            f"relativeTime={timestamp},"
            f"time={unix_time_ns}"
        )
        output.append(formatted_string)
    return output

def process_lines(lines, db):
    """
    Process a list of CAN message lines sequentially.

    Args:
        lines (list): List of CAN message lines as strings.
        db (cantools.database.Database): Loaded CAN database.

    Returns:
        list: A list of all formatted strings generated from the CAN messages.
    """
    output_strings = []
    for idx, line in enumerate(lines):
        try:
            parsed_strings = parse_can_message(line, db, idx + 1)  # Row number is 1-based
            output_strings.extend(parsed_strings)
        except Exception as e:
            log_error(f"Unexpected error: {str(e)}", line, idx + 1)
    return output_strings

def main():
    # Load CAN database
    dbc_file = 'testing_data/20240129 Gen5 CAN DB.dbc'
    try:
        db = cantools.database.load_file(dbc_file)
        print(f"Loaded DBC file: {dbc_file}")
    except Exception as e:
        log_error(f"Failed to load DBC file: {str(e)}", "N/A", "N/A")
        return

    # Read input lines
    input_file = 'testing_data/CanTraceJuly11.txt'
    try:
        with open(input_file, 'r') as f:
            lines = f.read().split('\n')
    except Exception as e:
        log_error(f"Failed to read input file: {str(e)}", "N/A", "N/A")
        return

    # Process lines and generate output
    output_strings = process_lines(lines, db)

    # Save to an output file
    output_file = 'testing_data/CanTraceJuly11_parsed.txt'
    try:
        with open(output_file, 'w') as f:
            for line in output_strings:
                f.write(line + '\n')
        print(f"Parsed data saved to {output_file}")
    except Exception as e:
        log_error(f"Failed to write to output file: {str(e)}", "N/A", "N/A")

if __name__ == "__main__":
    main()


Loaded DBC file: testing_data/20240129 Gen5 CAN DB.dbc
